In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 26 16:06:53 2020
@author: Fernando montenegro
"""
import numpy as np
import pandas as pd
from gurobipy import *
import xlrd
import csv


m= Model("vacunas")
#Conjuntos
K=14 #macro-divisiones de la región en estudio#listo
T=90 #Conjunto de periodos máximos a planificar #listo
P=5 #Conjunto de grupos poblacionales#listo
I=7 #Conjunto de instalaciones fijas para vacunación #listo
J=5 #Conjunto de centro móviles de vacunación#listo
low_pop=0.8
bar=68 #numero de barrios

L=[[1,2],[3,4],[5,6,7],[8,9,10,11],[12,13,14,15,16],[17,18,19,20,21,22,23,24,25,26,27,28,29,30,31],
  [32,33],[34,35,36,37,38,39,40,41,42],[43,44,45,46],[47,48,49,50,51,52],[53,54,55,56,57,58],[59],
  [60,61,62,63],[64,65,66,67,68]] #Conjunto de zonas de la macrodivisón#listo


MM=10000# m grande#listo
amb=[350 for i in range(T)]#costo de ambulancia#listo
#+np.random.uniform()*10 for i in range(T)

df=pd.read_excel('adyacencia.xlsx')
ndict={}
for i in range(1,bar+1):
    ndict[i]=df[i].tolist()

N={}
for i in range(1,bar+1):
    qq=[]
    for j in ndict[i]:
        if j>0:
            qq.append(j)
    N[i]=qq#matriz de adyacencia#listo
            

CK=[x for x in range(1,K+1)]
CT=[x for x in range(1,T+1)]
CP=[x for x in range(1,P+1)]
CI=[x for x in range(1,I+1)]
CJ=[x for x in range(1,J+1)]


df=pd.read_excel('pop.xlsx')


Po=[0 for a in range(1,bar+1)]
for i in range(1,bar+1):
    Po[i-1]=df[i].tolist()#demanda de vacunas en macrodivision segun grupos poblacionales#listo

    
df=pd.read_excel('H.xlsx')
H=[0 for a in range(1,I+1)]

for i in range(1,I+1):
    H[i-1]=df[i].tolist()#conjunto de zonas que se pueden atender en el mismo hospital

f1min=24379.995
f1max=115476.027
f2min=104650
f2max=157500

C=[122,100,89,78,78,78,78] #capacidad de vacunación de centros fijos
D=[200 for i in range(J)] #capacidad de vacunación de centros móviles
   
df=pd.read_excel('rt.xlsx')
rg=[0 for a in range(1,T+1)]
for i in range(1,T+1):
    rg[i-1]=df[i].tolist()#conjunto de zonas que se pueden atender en el mismo hospital

df=pd.read_excel('Abastecimiento.xlsx')
A=df.set_index('id')['value'].to_dict()


Academic license - for non-commercial use only - expires 2022-06-20
Using license file C:\Users\famd_\gurobi.lic


In [2]:

#gurobipy.gurobi.version()

In [3]:
#diccionario de parámetros

drg={}
for i in range(len(rg)):
    for j in range(len(rg[0])):
        drg[(i+1,j+1)]=rg[i][j]
  
dPo={}
for k in CK:  
    for i in L[k-1]:
       for p in CP:
            dPo[(i,p)]=Po[i-1][p-1]

dH={}
for i in range(len(H)):
    for j in range(len(H[0])):
        dH[(i+1,j+1)]=H[i][j]

damb={}
for k in CK:  
    for i in L[k-1]:
        damb[i]=amb[i-1]
        
dD=dict(zip(CJ,D))
dC=dict(zip(CI,C))


In [4]:
#variables
pqf={}#personas vacunadas por el equipo fijo en la macrozona
qf={}
xf={}#personas vacunadas por el equipo fijo en los barrios
pxf={}

for k in CK:
    for l in L[k-1]:
        for p in CP:
            for i in CI:
                qf[k,p,i]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="qf[%s,%s,%s]"%(k,p,i))
                for t in CT:
                    xf[l,p,i,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="xf[%s,%s,%s,%s]"%(l,p,i,t))


qm={}#personas vacunadas por el equipo movil en la macrozona
pqm={}
pxm={}
xm={}#personas vacunadas por el equipo movil en los barrios
v={}# indica qué población atiende el equipo movil
for k in CK:
    for p in CP:
        for j in CJ:
            qm[k,p,j]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="qm[%s,%s,%s]"%(k,p,j))
            for l in L[k-1]:
                for t in CT:
                    xm[l,p,j,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="xm[%s,%s,%s,%s]"%(l,p,j,t))  
                    v[l,j,t,p]=m.addVar(vtype=GRB.BINARY, lb=0 ,name="v[%s,%s,%s,%s]"%(l,j,t,p))

z={}# cantidad de vacunas asignadas a la población k
y={}# indica si el movil se ha posicionado en alguna de las zonas
for k in CK:
    for t in CT:
        z[k,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="z[%s,%s]"%(k,t)) 
        for r in L[k-1]:
            for j in CJ:    
                  y[r,j,t]=m.addVar(vtype=GRB.BINARY, lb=0 ,name="y[%s,%s,%s]"%(r,j,t))

sol={}
for t in CT:
    for p in CP:
        sol[t,p]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="sol[%s,%s]"%(t,p))


epsilon = 0.01
m.update()

#Restricciones
#2)calculamos las personas por macrozonas
#m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i,t) in xf )>=
 #       quicksum(qf[k,p,i] for (k,p,i) in qf )) 
#m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i,t) in xf )<=
 #       quicksum(qf[k,p,i] for (k,p,i) in qf )+epsilon)


#3)calculamos las personas por macrozonas
#m.addConstr(quicksum(xm[l,p,j,t] for (l,p,j,t) in xm )>=
  #      quicksum(qm[k,p,j] for (k,p,j) in qm))


#m.addConstr(quicksum(xm[l,p,j,t] for (l,p,j,t) in xm )<=
 #       quicksum(qm[k,p,j] for (k,p,j) in qm)+epsilon)

#6)Población de cada zona debe ser atendida 
for k in CK: 
    #5)Balance de vacunas desde la macroregión a la zonas (dentro de ellas)
    #m.addConstr(quicksum(qf[k,p,i] for (k1,p,i) in qf if k1==k )+
     #           quicksum(qm[k,p,j] for (k1,p,j) in qm if k1==k )<=
      #          quicksum(z[k,t] for (k1,t) in z if k1==k)+epsilon )

    #m.addConstr(quicksum(qf[k,p,i] for (k1,p,i) in qf if k1==k )+
     #           quicksum(qm[k,p,j] for (k1,p,j) in qm if k1==k )>=
       #         quicksum(z[k,t] for (k1,t) in z if k1==k) )

    
    for l in L[k-1]:
        for p in CP:
            m.addConstr(quicksum(xf[l,p,i,t] for (l1,p1,i,t) in 
                     xf if l1==l and p1==p)
            +quicksum(xm[l,p,j,t] for (l1,p1,j,t) in
                      xm if l1==l and p1==p)<=
                                    dPo[l,p])
            
    for l in L[k-1]:
        for p in CP:
            m.addConstr(quicksum(xf[l,p,i,t] for (l1,p1,i,t) in 
                     xf if l1==l and p1==p)
            +quicksum(xm[l,p,j,t] for (l1,p1,j,t) in
                      xm if l1==l and p1==p)>=
                                    dPo[l,p])

            
            
           
            
#8)capacidad de vacunatorios fijos
for i in CI:
    for t in CT:
        m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i1,t1) in xf if i1==i and t1==t)<=dC[i])  

#Restricciones de localización de vacunatorios móviles (máxima cobertura considerando zonas adyacentes)
#10)restricción
for t in CT:
    print('t: ', str(t), ' of ', str(len(CT)))
    #4)Balance de vacunas diarias entregadas desde centro de distribución a la macroregión (comunas en nuestro caso)
    #m.addConstr(quicksum(z[k,t] for k in CK if (k,t) in z)<= A[t])
    m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i,t1) in 
                     xf if t1==t)
            +quicksum(xm[l,p,j,t] for (l,p,j,t1) in
                      xm if t1==t)<=A[t])
    
    for j in CJ:
        #7)capacidad de vacunatorios móviles
        m.addConstr(quicksum(xm[l,p,j,t] for (l,p,j1,t1) in xm if j1==j and t1==t)<=dD[j])

        #se decide que se debem.addConstr(quicksum(y[r,j,t] for (r,j1,t1) in y if r in N[l] and j1==j and t1==t)>=v[l,j,t,p])                                
         #instalar a lo más en un barrio por día
        m.addConstr(quicksum(y[r,j,t] for (r,j1,t1) in y if j1==j and t1==t)<=1)
        for k in CK:
            for l in L[k-1]:
                    for p in CP:
                        m.addConstr(quicksum(y[r,j,t] for (r,j1,t1) in y if r in N[l] and j1==j and t1==t )>=v[l,j,t,p])                                
    #Se permite vacunar gente a través de un vacunatorio móvil siempre y cuando se encuentre en los barrios
    #cubiertos por él
    #11)restricción
                        m.addConstr(xm[l,p,j,t] <= v[l,j,t,p]*dPo[l,p])                   

#se busca permite que las personas se vacunen en zonas adyacentes a sus barrios
for t in CT:
    for p in CP:
        # suma de la función objetivo
        m.addConstr(quicksum(xm[l,p,j,t] for (l,p1,j,t1) in xm if p1==p and t1==t) + 
                    quicksum(xf[s,p,i,t] for (s,p1,i,t1) in xf if p1==p and t1==t)==sol[t,p])
        for k in CK:
            for l in L[k-1]:
                    for i in CI:
                        m.addConstr(xf[l,p,i,t] <= dH[i,l]*10000)

#restricción para que los grupos riesgosos no sean vacunados en los cesfams
m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i,t) in xf if p==1)==0)

fo1=quicksum(sol[t,p]*drg[t,p] for (t,p) in sol) #función objetivo1  
fo2=quicksum(damb[j]*y[l,j,t]  for (l,j,t) in y)#función objetivo2

#Model.write('scenario_1.lp')
m.update()

fn1=(fo1-f1min)/(f1max-f1min)
fn2=(fo2-f2min)/(f2max-f2min)

setSol = {}
count = 0
aa=[9.8]
#for mm in range(2,10,2):
for mm in aa:    
    jjj=mm/10
    if count > 0: 
        for k in CK:
            for l in L[k-1]:
                for p in CP:
                    for i in CI:
                        #qf[k,p,i].start=qf[k,p,i].x
                        for t in CT:
                            xf[l,p,i,t].start=xf[l,p,i,t].x
        for k in CK:
            for p in CP:
                for j in CJ:
                    #qm[k,p,j].start=qm[k,p,j].x
                    for l in L[k-1]:
                        for t in CT:
                            xm[l,p,j,t].start=xm[l,p,j,t].x
                            v[l,j,t,p].start=v[l,j,t,p].x

        for k in CK:
            for t in CT:
                #z[k,t].start=z[k,t].x
                for r in L[k-1]:
                    for j in CJ:    
                          y[r,j,t].start=y[r,j,t].x

    
    #m.Params.Method=2
    m.Params.Heuristics =0.5
    m.Params.ObjNumber  = 2
    #m.Params.timelimit=14400
    m.Params.timelimit=1400
    m.Params.MIPFocus=3
    m.Params.ImproveStartTime=7200
    m.Params.PoolSearchMode = 2
    m.Params.Presolve = 1
    #m.Params.MIPGap=0.001
    #m.Params.Cuts=3
    
    m.ModelSense=1 #minimization 
    #m.tune()     
    m.setObjectiveN(fo1, 0, weight=jjj, name="f1")    
    m.setObjectiveN(fo2, 1, weight=1-jjj, name="f2")

    print('alpha: ', jjj)


    m.Params.LogFile=str(jjj)+'_ex.log'
    m.update()

    m.optimize()
    Time_r=m.Runtime
    m.write(str(jjj)+"_exp1.sol")
    count = count + 1
    
    #m.discardMultiobjEnvs()
    
    nSolutions = m.SolCount
    population=sum(sol[t,p].x for (t,p) in sol)

    #m.status = GRB.Status.OPTIMAL    
    #if m.status == GRB.Status.OPTIMAL:    
    if True:
        # Query number of multiple objectives, and number of solutions
        nSolutions  = m.SolCount
        nObjectives = m.NumObj

        for s in range(nSolutions):
         # Set which solution we will query from now on
          m.params.SolutionNumber = s
        # Print objective value of this solution in each objective
        # print('Solution', s, ':', end='')
        for o in range(nObjectives):
        # Set which objective we will query
            m.params.ObjNumber = o
        # Query the o-th objective value
            #print(' ',m.ObjNVal, end='')


        nObjectives = m.NumObj
        #print('Problem has', nObjectives, 'objectives')
        for o in range(nObjectives):
        # Set which objective we will query
            m.params.ObjNumber = o

        # query the full vector of the o-th solution
        #assert m.Status == GRB.Status.OPTIMAL

        # Query number of multiple objectives, and number of solutions
        nSolutions  = m.SolCount
        nObjectives = m.NumObj

    solutions = []

    gaps=[]
    soluciones={}
    sal=[]
    sal = np.zeros((nSolutions, nObjectives))
    for s in range(nSolutions):
    # Set which solution we will query from now on
        m.params.SolutionNumber = s

        # Print objective value of this solution in each objective
        #print('Solution', s, ':', end='')
        for o in range(nObjectives):
            i=o
        # Set which objective we will query
            m.params.ObjNumber = o
        # Query the o-th objective value
            soluciones[(s+1,i+1)]=m.ObjNVal 
            sal[s-1][i-1]=m.ObjNVal

    setSol[jjj]=soluciones
    #import Outputs
    pt1="plt1"+str(jjj)+".png"
    pt2="plt2"+str(jjj)+".png"
    pt3="plt3"+str(jjj)+".png"
    #calculo de atención por hospitales
    amb1={}
    amb2={}
    amb3={}
    amb4={}
    amb5={}
    amb6={}
    amb7={}
    amb8={}
    amb9={}
    amb10={}


    for tt in range(1,T+1):
        amb1[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==1)),2)
        amb2[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==2)),2)
        amb3[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==3)),2)
        amb4[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==4)),2)
        amb5[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==5)),2)
        amb6[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==6)),2)
        amb7[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==7)),2)
        amb8[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==8)),2)
        amb9[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==9)),2)
        amb10[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==10)),2)


    import pandas as pd
    data2 = pd.DataFrame(columns=( 'day','ab1','ab2','ab3','ab4','ab5','ab6','ab7','ab8','ab9','ab10'))

    for t in CT:
        data2.loc[t]=[t,amb1[t],amb2[t],amb3[t],amb4[t],amb5[t],amb6[t],amb7[t],amb8[t],amb9[t],amb10[t]] 
    


    dia_xa={}
    dia_xb={}
    dia_xc={}
    dia_xd={}
    dia_xe={}

    for tt in range(1,T+1):
        dia_xa[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==1))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==1)),2)
        dia_xb[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==2))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==2)),2)
        dia_xc[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==3))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==3)),2)
        dia_xd[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==4)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==4))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==4)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==4)),2)
        dia_xe[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==5))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==5)),2)


    import pandas as pd
    data = pd.DataFrame(columns=( 'day','A', 'B', 'C', 'D', 'E'))

    for t in CT:
        data.loc[t]=[t,dia_xa[t],dia_xb[t],dia_xc[t],dia_xd[t],dia_xe[t]] 

    dia_q1={}
    dia_q2={}
    dia_q3={}
    dia_q4={}
    dia_q5={}
    dia_q6={}
    dia_q7={}
    dia_q8={}
    dia_q9={}
    dia_q10={}
    dia_q11={}
    dia_q12={}
    dia_q13={}
    dia_q14={}

    for tt in range(1,T+1):
        dia_q1[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<=2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<=2))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<=2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<=2)),2)
        dia_q2[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<2 and l<=4)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<2 and l<=4))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<2 and l<=5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<2 and l<=4)),2)
        dia_q3[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<4 and l<=7)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<4 and l<=7))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<4 and l<=7)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<4 and l<=7)),2)
        dia_q4[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<7 and l<=11)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<7 and l<=11))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<7 and l<=11)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<7 and l<=11)),2)
        dia_q5[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<11 and l<=16)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<11 and l<=16))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<11 and l<=16)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<11 and l<=16)),2)
        dia_q6[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<16 and l<=31)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<16 and l<=31))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<16 and l<=31)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<16 and l<=31)),2)
        dia_q7[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<31 and l<=33)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<31 and l<=33))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<31 and l<=33)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<31 and l<=33)),2)
        dia_q8[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<33 and l<=42)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<33 and l<=42))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<33 and l<=42)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<33 and l<=42)),2)
        dia_q9[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<42 and l<=46)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<42 and l<=46))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<42 and l<=46)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<42 and l<=46)),2)
        dia_q10[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<46 and l<=52)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<46 and l<=52))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<46 and l<=52)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<46 and l<=52)),2)
        dia_q11[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<52 and l<=58)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<52 and l<=58))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<52 and l<=58)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<52 and l<=58)),2)
        dia_q12[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<58 and l<=59)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<58 and l<=59))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<58 and l<=59)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<58 and l<=59)),2)
        dia_q13[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<59 and l<=63)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<59 and l<=63))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<59 and l<=63)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<59 and l<=63)),2)
        dia_q14[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<63)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<63))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<63)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<63)),2)

    import pandas as pd
    data1 = pd.DataFrame(columns=( 'day','k1', 'k2', 'k3','k4','k5','k6','k7','k8','k9','k10','k11','k12','k13','k14'))

    for t in CT:
        data1.loc[t]=[t,dia_q1[t],dia_q2[t],dia_q3[t],dia_q4[t],dia_q5[t],dia_q6[t],dia_q7[t],dia_q8[t],dia_q9[t],dia_q10[t],dia_q11[t],dia_q12[t],dia_q13[t],dia_q14[t]] 
    #plot grupos de riesgos por día
    import seaborn as sns
    import matplotlib.pyplot as plt
    import pandas as pd
    from matplotlib import pyplot as pl
    import numpy as np
    fig = plt.figure()
    x = data['day']
    y1 = data['A']
    y2 = data['B']
    y3 = data['C']
    y4 = data['D']
    y5 = data['E']

    pl.plot(x, y1, 'k-', label = 'Population group: A')
    pl.legend()
    pl.fill_between(x, y1-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red')
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue')
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
    pl.plot(x, y4, 'k-', label = 'Population group: D', color = 'green')
    pl.fill_between(x, y4-0, alpha = 0.15, color='green')
    pl.plot(x, y5, 'k-', label = 'Population group: E', color = 'cyan')
    pl.fill_between(x, y5-0, alpha = 0.15, color='cyan')

    plt.ylim(0,100)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=15) 
    pl.rc('ytick', labelsize=15) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.savefig(pt1)
    pl.show()
    

    fig = plt.figure()
    x = data['day']
    y1 = data['A']
    y2 = data['B']
    y3 = data['C']
    y4 = data['D']
    y5 = data['E']

    pl.plot(x, y1, 'k-', label = 'Population group: A')
    pl.legend()
    pl.fill_between(x, y1-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red')
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue')
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
    pl.plot(x, y4, 'k-', label = 'Population group: D', color = 'green')
    pl.fill_between(x, y4-0, alpha = 0.15, color='green')
    pl.plot(x, y5, 'k-', label = 'Population group: E', color = 'cyan')
    pl.fill_between(x, y5-0, alpha = 0.15, color='cyan')

    plt.ylim(0,100)
    plt.xlim(0,90)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=15) 
    pl.rc('ytick', labelsize=15) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.savefig(pt2)
    pl.show()
    

    fig = plt.figure()
    x = data1['day']
    y1 = data1['k1']
    y2 = data1['k2']
    y3 = data1['k3']
    y4 = data1['k4']
    y5 = data1['k5']
    y6 = data1['k6']
    y7 = data1['k7']
    y8 = data1['k8']
    y9 = data1['k9']
    y10 = data1['k10']
    y11 = data1['k11']
    y12 = data1['k12']
    y13 = data1['k13']
    y14 = data1['k14']


    pl.plot(x, y1, 'k-', label = 'Macrozone 1',marker="s")
    pl.legend()
    pl.fill_between(x, y1-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Macrozone 2', color = 'red',marker="o")
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Macrozone 3', color = 'blue',marker="X")
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
    pl.plot(x, y4, 'k-', label = 'Macrozone 4', color = 'cyan',marker="h")
    pl.fill_between(x, y4-0, alpha = 0.15, color='cyan')
    pl.plot(x, y5, 'k-', label = 'Macrozone 5', color = 'blue',marker="H")
    pl.fill_between(x, y5-0, alpha = 0.15, color='blue')
    pl.plot(x, y6, 'k-', label = 'Macrozone 6', color = 'magenta',marker="p")
    pl.fill_between(x, y6-0, alpha = 0.15, color='magenta')
    pl.plot(x, y7, 'k-', label = 'Macrozone 7', color = 'blue',marker="*")
    pl.fill_between(x, y7-0, alpha = 0.15, color='blue')
    pl.plot(x, y8, 'k-', label = 'Macrozone 8', color = 'yellow',marker="X")
    pl.fill_between(x, y8-0, alpha = 0.15, color='yellow')
    pl.plot(x, y9, 'k-', label = 'Macrozone 9', color = 'cyan',marker="+")
    pl.fill_between(x, y9-0, alpha = 0.15, color='cyan')
    pl.plot(x, y10, 'k-', label = 'Macrozone 10', color = 'magenta',marker="d")
    pl.fill_between(x, y10-0, alpha = 0.15, color='magenta')
    pl.plot(x, y11, 'k-', label = 'Macrozone 11', color = 'yellow',marker="D")
    pl.fill_between(x, y11-0, alpha = 0.15, color='yellow')
    pl.plot(x, y12, 'k-', label = 'Macrozone 12', color = 'blue',marker="<")
    pl.fill_between(x, y12-0, alpha = 0.15, color='blue')
    pl.plot(x, y13, 'k-', label = 'Macrozone 13', color = 'red',marker=">")
    pl.fill_between(x, y13-0, alpha = 0.15, color='red')
    pl.plot(x, y14, 'k-', label = 'Macrozone 14', color = 'cyan',marker="^")
    pl.fill_between(x, y14-0, alpha = 0.15, color='cyan')

    plt.ylim(0,100)
    plt.xlim(0,90)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=10) 
    pl.rc('ytick', labelsize=10) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='lower right', shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.savefig(pt3)
    pl.show()
    
    #calculo de atención por hospitales
    hos1={}
    hos2={}
    hos3={}
    hos4={}
    hos5={}
    hos6={}
    hos7={}


    for tt in range(1,T+1):
        hos1[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==1)),0)
        hos2[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==2)),0)
        hos3[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==3)),0)
        hos4[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==4)),0)
        hos5[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==5)),0)
        hos6[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==6)),0)
        hos7[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==7)),0)
    import pandas as pd
    data2 = pd.DataFrame(columns=( 'day','h1', 'h2', 'h3','h4','h5','h6','h7'))

    for t in range(1,T+1):
        data2.loc[t]=[t,hos1[t],hos2[t],hos3[t],hos4[t],hos5[t],hos6[t],hos7[t]] 

    from openpyxl import Workbook

    #workbook_name="Hoja1.xlsx"
    #wb = load_workbook(workbook_name)
    wb = Workbook()
    ws1 = wb.create_sheet("F.O's")
    ws2 = wb.create_sheet("Plan_vacunación")
    ws3 = wb.create_sheet("Plan_móvil")
    ws4 = wb.create_sheet("Plan_fijo")
    ws5 = wb.create_sheet("Tiempo_Gap")


    #guardando el valor de las f.o's en excel
    for i in range(len(sal)):
        for j in range(len(sal[0])):
                ws1.cell(row = 1, column = 1, value = "f1")
                ws1.cell(row = i+2, column = 1, value = soluciones[i+1,1])
                ws1.cell(row = 1, column = 2, value = "f2")
                ws1.cell(row = i+2, column = 2, value = soluciones[i+1,2])

    #plan de vacunación
    for t in range(1,T+1):
        ws2.cell(row = 1, column = 1, value = "día")
        ws2.cell(row = t+1, column = 1, value = t)
        ws2.cell(row = 1, column = 2, value = "CF1")
        ws2.cell(row = t+1, column = 2, value = hos1[t])
        ws2.cell(row = 1, column = 3, value = "CF2")
        ws2.cell(row = t+1, column = 3, value = hos2[t])
        ws2.cell(row = 1, column = 4, value = "CF3")
        ws2.cell(row = t+1, column = 4, value = hos3[t])
        ws2.cell(row = 1, column = 5, value = "CF4")
        ws2.cell(row = t+1, column = 5, value = hos4[t])    
        ws2.cell(row = 1, column = 6, value = "CF5")
        ws2.cell(row = t+1, column = 6, value = hos5[t])    
        ws2.cell(row = 1, column = 7, value = "CF6")
        ws2.cell(row = t+1, column = 7, value = hos6[t])
        ws2.cell(row = 1, column = 8, value = "CF7")
        ws2.cell(row = t+1, column = 8, value = hos7[t])    

        ws2.cell(row = 1, column = 9, value = "CM1")
        ws2.cell(row = t+1, column = 9, value = amb1[t])             
        ws2.cell(row = 1, column = 10, value = "CM2") 
        ws2.cell(row = t+1, column = 10, value = amb2[t]) 
        ws2.cell(row = 1, column = 11, value = "CM3")
        ws2.cell(row = t+1, column = 11, value = amb3[t]) 
        ws2.cell(row = 1, column = 12, value = "CM4")             
        ws2.cell(row = t+1, column = 12, value = amb4[t] )    
        ws2.cell(row = 1, column = 13, value = "CM5") 
        ws2.cell(row = t+1, column = 13, value = amb5[t])
        ws2.cell(row = 1, column = 14, value = "CM6")
        ws2.cell(row = t+1, column = 14, value = amb6[t])
        ws2.cell(row = 1, column = 15, value = "CM7")
        ws2.cell(row = t+1, column = 15, value = amb7[t])
        ws2.cell(row = 1, column = 16, value = "CM8")
        ws2.cell(row = t+1, column = 16, value = amb8[t])
        ws2.cell(row = 1, column = 17, value = "CM9")
        ws2.cell(row = t+1, column = 17, value = amb9[t])    
        ws2.cell(row = 1, column = 18, value = "CM10")
        ws2.cell(row = t+1, column = 18, value = amb10[t])    


    #plan de vacunación móvil
    cont=1
    for k in CK:
        for r in L[k-1]:
            for j in CJ:    
                for t in CT:
                    if y[r,j,t].x>0:
                        cont=cont+1   
                        ws3.cell(row = 1, column = 1, value = "Día")
                        ws3.cell(row = 1, column = 2, value = "Centro Móvil")
                        ws3.cell(row = 1, column = 3, value = "Se instala en") 
                        ws3.cell(row = cont+1, column = 1, value = t)
                        ws3.cell(row = cont+1, column = 2, value = j)
                        ws3.cell(row = cont+1, column = 3, value = r)
    cont=1
    for k in CK:
        for l in L[k-1]:
            for j in CJ:
                for t in CT:
                    for p in CP:
                        if v[l,j,t,p].x>0:
                            cont=cont+1

                            ws3.cell(row = 1, column = 4, value = "Día")
                            ws3.cell(row = 1, column = 5, value = "Centro Móvil")
                            ws3.cell(row = 1, column = 6, value = "Barrio")
                            ws3.cell(row = 1, column = 7, value = "Grupo")
                            ws3.cell(row = 1, column = 8, value = "Atiende")

                            ws3.cell(row = cont+1, column = 4, value = t)
                            ws3.cell(row = cont+1, column = 5, value = j)
                            ws3.cell(row = cont+1, column = 6, value = l)
                            ws3.cell(row = cont+1, column = 7, value = p)
                            ws3.cell(row = cont+1, column = 8, value = v[l,j,t,p].x)

    cont=1
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for j in CJ:
                   for t in CT:
                        if xm[l,p,j,t].x>0:
                            cont=cont+1
                            ws3.cell(row = 1, column = 9, value = "Día")
                            ws3.cell(row = 1, column = 10, value = "Centro Móvil")
                            ws3.cell(row = 1, column = 11, value = "Barrio")
                            ws3.cell(row = 1, column = 12, value = "Grupo")
                            ws3.cell(row = 1, column = 13, value = "Cantidad")
                            
                            ws3.cell(row = cont+1, column = 9, value = t)
                            ws3.cell(row = cont+1, column = 10, value = j)
                            ws3.cell(row = cont+1, column = 11, value = l)
                            ws3.cell(row = cont+1, column = 12, value = p)
                            ws3.cell(row = cont+1, column = 13, value = xm[l,p,j,t].x)
                            
    cont=1
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for i in CI:
                    for t in CT:
                        if xf[l,p,i,t].x>0:
                            cont=cont+1
                            ws4.cell(row = 1, column = 1, value = "Día")
                            ws4.cell(row = 1, column = 2, value = "CF")
                            ws4.cell(row = 1, column = 3, value = "Barrio")
                            ws4.cell(row = 1, column = 4, value = "Grupo")
                            ws4.cell(row = 1, column = 5, value = "Cantidad")

                            ws4.cell(row = cont+1, column = 1, value = t)
                            ws4.cell(row = cont+1, column = 2, value = i)
                            ws4.cell(row = cont+1, column = 3, value = l)
                            ws4.cell(row = cont+1, column = 4, value = p)
                            ws4.cell(row = cont+1, column = 5, value = xf[l,p,i,t].x)

    ws5.cell(row = 2, column = 2, value = "Tiempo")
    ws5.cell(row = 3, column = 2, value = Time_r)
    # Save the file
    namexl="output"+str(jjj)+".xlsx"
    wb.save(namexl)




t:  1  of  90
t:  2  of  90
t:  3  of  90
t:  4  of  90
t:  5  of  90
t:  6  of  90
t:  7  of  90
t:  8  of  90
t:  9  of  90
t:  10  of  90
t:  11  of  90
t:  12  of  90
t:  13  of  90
t:  14  of  90
t:  15  of  90
t:  16  of  90
t:  17  of  90
t:  18  of  90
t:  19  of  90
t:  20  of  90
t:  21  of  90
t:  22  of  90
t:  23  of  90
t:  24  of  90
t:  25  of  90
t:  26  of  90
t:  27  of  90
t:  28  of  90
t:  29  of  90
t:  30  of  90
t:  31  of  90
t:  32  of  90
t:  33  of  90
t:  34  of  90
t:  35  of  90
t:  36  of  90
t:  37  of  90
t:  38  of  90
t:  39  of  90
t:  40  of  90
t:  41  of  90
t:  42  of  90
t:  43  of  90
t:  44  of  90
t:  45  of  90
t:  46  of  90
t:  47  of  90
t:  48  of  90
t:  49  of  90
t:  50  of  90
t:  51  of  90
t:  52  of  90
t:  53  of  90
t:  54  of  90
t:  55  of  90
t:  56  of  90
t:  57  of  90
t:  58  of  90
t:  59  of  90
t:  60  of  90
t:  61  of  90
t:  62  of  90
t:  63  of  90
t:  64  of  90
t:  65  of  90
t:  66  of  90
t:  67  of  90
t:  

  248689    6.8720977e+04   0.000000e+00   1.498014e+04    355s
  251727    6.8716679e+04   0.000000e+00   5.660926e+03    360s
  254548    6.8714013e+04   0.000000e+00   7.302068e+03    365s
  257369    6.8710929e+04   0.000000e+00   6.032976e+03    370s
  260190    6.8707944e+04   0.000000e+00   4.769506e+03    375s
  263011    6.8705883e+04   0.000000e+00   3.763836e+04    380s
  266049    6.8703278e+04   0.000000e+00   1.983328e+03    385s
  268870    6.8699921e+04   0.000000e+00   6.045473e+04    390s
  271691    6.8696559e+04   0.000000e+00   8.505736e+03    395s
  274295    6.8693830e+04   0.000000e+00   4.682991e+03    400s
  277333    6.8691426e+04   0.000000e+00   1.492022e+03    405s
  279937    6.8689365e+04   0.000000e+00   2.251764e+03    410s
  282107    6.8687502e+04   0.000000e+00   2.229028e+03    415s
  284711    6.8685746e+04   0.000000e+00   2.529277e+03    420s
  287315    6.8684263e+04   0.000000e+00   1.426019e+04    425s
  289702    6.8683104e+04   0.000000e+00

GurobiError: Unable to retrieve attribute 'X'

In [ ]:
F1 = []
F2 = []
for e in setSol.keys():
   # print(setSol[e].keys())
    #f = 0
    for f in range(len(setSol[e].keys())):
        #print('alpha: ', e)
        print('beta: ', list(setSol[e].keys())[f])
        #print(setSol[e])
        #print(setSol[e].keys()[f])
        if '1)' in str(list(setSol[e].keys())[f]):
            print('f1: ', setSol[e][list(setSol[e].keys())[f]])
            F1.append(setSol[e][list(setSol[e].keys())[f]])
        else:
            #print('f2: ', setSol[e][list(setSol[e].keys())[f]])
            F2.append(setSol[e][list(setSol[e].keys())[f]])

In [ ]:
print(soluciones)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(F1, F2,marker='x', color="black")
plt.xlabel('$f_{1}^{*}$')
plt.ylabel('$f_{2}^{*}$')
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
totalmacro={}
for ii in CT:
    totalmacro[ii]=sum(z[k,ii].x for k in CK)
print(totalmacro)

In [ ]:
pas={} # se cálcula la cantidad de personas efectivmente vacunadas tanto en hospitales como ambulancias
print('Cantidad de personas vacunadas según grupo de riesgo' )
for k in CK:
    for l in L[k-1]:
        for p in CP:
            pas[l,p]=sum(xf[l,p,i,t].x for (l1,p1,i,t) in xf if l1==l and p1==p)+ sum(xm[l,p,j,t].x for (l1,p1,j,t) in xm if l1==l and p1==p)

#for p in CP:
 #   for k in CK:
  #      for l in L[k-1]:   
            #print('Barrio:',l, ', Paciente:',p, 'Cantidad:', float(pas[l,p]))

totalbarrio={}#vacunados por barrios
totaldembar={}#demanda por barrios
nivelser={}#nivel de servicio
totalvac=0 #vacunas entregadas al sistema 
for t in CT:
    totalvac=totalvac+ A[t]

totalpob=0
for k in CK:  
    for i in L[k-1]:
       for p in CP:
        totalpob=totalpob+dPo[(i,p)] #cálculamos la cantidad de personas dísponibles a vacunar
print('Vacunas totales :', totalvac )
print('Población total:', totalpob)
totalvacunados=0
totalvacunados=sum(pas[l,p] for (l,p) in pas) #calculamos las personas realmente vacunadas
print('Personas vacunadas: ', totalvacunados)

print('Porcentaje de cobertura campaña:', round(100*(totalvac/totalpob),2),'%')
for k in CK:
    for l in L[k-1]:
        totalbarrio[l]=sum(pas[l,p] for (l1,p) in pas if l1==l)
        totaldembar[l]=sum(dPo[l,p] for (l1,p) in dPo if l1==l)
        print('Porcentaje vacunados por barrio',l,':',round(100*(totalbarrio[l]/totaldembar[l]),2),'%')
        nivelser[l]=round(100*(totalbarrio[l]/totaldembar[l]),2)

import pandas as pd
data0 = pd.DataFrame(columns=( 'f1','f2'))

for i in range(len(sal)):
    for j in range(len(sal[0])):
        data0.loc[i]=[ soluciones[i+1,1],soluciones[i+1,2]] 

print(data0)